In [1]:
#    pip install pymdptoolbox
#Pour ce installez la (si ce n'est dejà fait) et importez la toolbox et les exemples

In [2]:
pip install pymdptoolbox

  Preparing metadata (setup.py) ... done
  Created wheel for pymdptoolbox: filename=pymdptoolbox-4.0b3-py3-none-any.whl size=25656 sha256=9fab7a82cb5fac0f50c4d7b62007599c655b7a02d599bfa8ffc5ccc6850817da
  Stored in directory: /root/.cache/pip/wheels/2b/e7/c7/d7abf9e309f3573a934fed2750c70bd75d9e9d901f7f16e183
Successfully built pymdptoolbox


In [3]:
pip install mdptoolbox

ERROR: Could not find a version that satisfies the requirement mdptoolbox (from versions: none)
ERROR: No matching distribution found for mdptoolbox


In [4]:
import mdptoolbox, mdptoolbox.example, mdptoolbox.util
import numpy as np

# Le Robot Netoyeur

A rendre pour le  15 novembre ( rendre votre note book  plus un petit rapport decrivant votre MDP, incluant un shema du mdp)

On considère un robot ménager. Il peut se trouver soit dans le salon, soit à sa station de charge, soit être à court de batterie. Le salon peut être propre ou sale. Il y a donc cinq états : LD (dans le salon, sale), LC (dans le salon, propre), CD (au chargeur, sale), CC (au chargeur, propre), O (hors tension).


####  
Dans le salon, le robot peut choisir de nettoyer ou de charger.

Lorsqu'il se trouve dans la station de charge, le robot peut choisir de nettoyer ou de continuer à charger.

Lorsqu'il est hors tension,  chacune des deux actions (nettoyer, charger) aboutit au même résultat : rester hors tension.

####  

Le reward pour être dans un état propre est noté rc

Le reward pour être dans un état sale est  noté rd, et tel que  rd < rc  

Le reward pour être hors tension est noté costcrash - il doit être inférieur ou égal à rd ; on peut le fixer égal à rd dans un premier temps (les salond deviennent sales  quand on ne les nettoie pas)


####  

     
Lorsque le robot décide de nettoyer,
* s'il est dans le salon, il risque de tomber en panne d'électricité avec une probabilité e
* s'il se trouve dans la station de charge, il ne risque pas de tomber en panne d'électricité   
* le nettoyage d'un sol propre le laisse propre
* Le nettoyage d'un sol sale peut parfois échouer (même si la batterie est en bon état) - soit eps la probabilité d'échec du nettoyage.

Lorsque le robot décide de se recharger, l'action de rechargement l'amène toujours à la station de recharge. Lorsque le robot charge,   peut passer de propre à sale avec une probabilité pDust - et bien sur, si le salon est déjà sale, il reste sale lorsque le robot est en charge


# **Modélisation**


#### Objectif

Modéliser le problème par un MDP (horizon infini)  et décrire votre modèle en détail

In [33]:
def create_P_R(rd, rc, costcrash, e, eps, pDust):
  P = np.zeros((2, 5, 5))
  R = np.zeros((5, 2))
  P_nettoyer = [
            [eps, 1-(eps+e), 0.0, 0.0, e],
            [0.0, 1-e, 0.0, 0.0, e],
            [eps, 1-eps, 0.0, 0.0, 0.0],
            [0, 1, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1]
  ]
  P_charger = [
            [0.0, 0.0, 1, 0.0, 0.0],
            [0.0, 0.0, 0.0, 1, 0.0],
            [0.0, 0.0, 1, 0.0, 0.0],
            [0.0, 0.0, pDust, 1-pDust, 0.0],
            [0.0, 0.0, 0.0, 0.0, 1]
  ]

  P[0] = P_nettoyer
  P[1] = P_charger

  R[0] = [rd , rd]
  R[1] = [rc , rc]
  R[2] = [rd , rd]
  R[3] = [rc , rc]
  R[4] = [costcrash , costcrash]

  # Check
  mdptoolbox.util.check(P, R)

  return P, R



  # Définir les états (LD, LC, CD, CC, O)
etats = ('LD', 'LC', 'CD', 'CC', 'O')

# Définir les actions (nettoyer et charger)
actions = ('nettoyer', 'charger')

def display(policy):
    acts = [actions[i] for i in policy]

    print("Policy:")
    print("LD: {}".format(acts[0]))
    print("LC: {}".format(acts[1]))
    print("CD: {}".format(acts[2]))
    print("CC: {}".format(acts[3]))
    print("0: {}".format(acts[4]))

def run(rd, rc, costcrash, e, eps, pDust):
    if (e+eps)>1:
      print("Mauvaises valeurs pour e: {} et eps: {} la somme doit être <= 1".format(e, eps))
    elif (pDust > 1):
      print("Mauvaises valeurs pour pDust: {} la valeur doit être <= 1".format(pDust))
    elif (pDust < 0) or (e < 0) or (eps < 0):
      print("Mauvaises valeurs pour e: {}, eps: {} et pDust: {} les probabilités sont positives. <= 1".format(e, eps, pDust))
    elif (rd >= rc) or (rd != costcrash):
      print("Mauvaises valeurs pour rd: {}, rc: {} et costcrash: {} => rc > rd et rd = costcrash.".format(rd, rc, costcrash))
    else:
      P,R = create_P_R(rd, rc, costcrash, e, eps, pDust)
      #pi = mdptoolbox.mdp.ValueIterationGS(P, R, 0.9, 0.01)
      pi = mdptoolbox.mdp.PolicyIteration(P, R, 0.9)

      #pi.setVerbose()
      pi.run()
      display(pi.policy)
      pi.V
      del pi
costcrash=-10


Déterminer la meilleure politique.

* Lorsque la probabilité d'être à court de batterie est élevée
* Lorsqu'elle est faible

(expliquer les résultats)

# **Proba Court Batterie élevée**

In [6]:
e, eps, pDust = 0.95, 0.05, 0.5
rd, rc=-10, 10
print("Lorsque la probabilité d'être à court de batterie est élevée")
print("rd = {}, rc = {}, costcrash = {}, e = {}, eps = {}, pDust = {},".format(rd, rc, costcrash, e, eps, pDust))
run(rd, rc, costcrash, e, eps, pDust)

Lorsque la probabilité d'être à court de batterie est élevée
rd = -10, rc = 10, costcrash = -10, e = 0.95, eps = 0.05, pDust = 0.5,
Policy:
LD: charger
LC: charger
CD: nettoyer
CC: nettoyer
0: nettoyer


 La politique choisit de charger le robot dans les états "LD" (dans le salon, sale) et "LC" (dans le salon, propre), puis de charger à nouveau deux fois.
Ensuite, elle choisit de nettoyer trois fois consécutives dans les états "CD"(au chargeur, Sale), "CC"(au chargeur, propre) et "O"(Hors-Tension).

On explique ça par le fait que la politique évite le nettoyage dans le salon lorsqu'il est sale, pour minimiser le risque de panne d'électricité pendant le nettoyage.

# **Proba Court Batterie Faible**

In [22]:
e, eps, pDust = 0.3, 0.05, 0.5
rd, rc=-10, 10
print("La proba Court batterie Faible")
print("rd = {}, rc = {}, costcrash = {}, e = {}, eps = {}, pDust = {},".format(rd, rc, costcrash, e, eps, pDust))

run(rd, rc, costcrash, e, eps, pDust)

La proba Court batterie Faible
rd = -10, rc = 10, costcrash = -10, e = 0.3, eps = 0.05, pDust = 0.5,
Policy:
LD: charger
LC: charger
CD: nettoyer
CC: nettoyer
0: nettoyer


#Question :
Quelle politique est choisie si le robot a une très bonne batterie (c'est-à-dire que la probabilité d'être à court de charge est très faible) ?


Quelle politique est choisie si le salon se salit rapidement (lorsque la probabilité de poussière augmente) ?





In [31]:
e, eps, pDust = 0.05, 0.05, 0.3
rd, rc=-10, 10
print("La proba Court de charge est trés faible")
print("rd = {}, rc = {}, costcrash = {}, e = {}, eps = {}, pDust = {},".format(rd, rc, costcrash, e, eps, pDust))

run(rd, rc, costcrash, e, eps, pDust)

La proba Court de charge est trés faible
rd = -10, rc = 10, costcrash = -10, e = 0.05, eps = 0.05, pDust = 0.3,
Policy:
LD: nettoyer
LC: charger
CD: nettoyer
CC: nettoyer
0: nettoyer


# **Proba pDust augmente**

In [23]:
e, eps, pDust = 0.9, 0.05, 0.95
rd, rc=-10, 10
print("lorsque la probabilité de poussière augmente")
print("rd = {}, rc = {}, costcrash = {}, e = {}, eps = {}, pDust = {},".format(rd, rc, costcrash, e, eps, pDust))

run(rd, rc, costcrash, e, eps, pDust)

lorsque la probabilité de poussière augmente
rd = -10, rc = 10, costcrash = -10, e = 0.9, eps = 0.05, pDust = 0.95,
Policy:
LD: charger
LC: charger
CD: nettoyer
CC: nettoyer
0: nettoyer


In [28]:
e, eps, pDust = 0.1, 0.05, 0.95
rd, rc=-10, 10
print("lorsque la probabilité de poussière augmente et e=0.1")
print("rd = {}, rc = {}, costcrash = {}, e = {}, eps = {}, pDust = {},".format(rd, rc, costcrash, e, eps, pDust))

run(rd, rc, costcrash, e, eps, pDust)

lorsque la probabilité de poussière augmente et e=0.1
rd = -10, rc = 10, costcrash = -10, e = 0.1, eps = 0.05, pDust = 0.95,
Policy:
LD: charger
LC: charger
CD: nettoyer
CC: nettoyer
0: nettoyer


In [27]:
e, eps, pDust = 0.35, 0.65, 0.95
rd, rc=-10, 10
print("lorsque la probabilité de poussière augmente et e=0.35")
print("rd = {}, rc = {}, costcrash = {}, e = {}, eps = {}, pDust = {},".format(rd, rc, costcrash, e, eps, pDust))

run(rd, rc, costcrash, e, eps, pDust)

lorsque la probabilité de poussière augmente et e=0.35
rd = -10, rc = 10, costcrash = -10, e = 0.35, eps = 0.65, pDust = 0.95,
Policy:
LD: charger
LC: charger
CD: nettoyer
CC: nettoyer
0: nettoyer






Coller votre programme python ici:



# 3. Amelioration du modele

Ameliorez votre modèle de manière à prendre en considération plusieurs niveaux de batterie (élevé, moyen, faible  par exemple). Collez votre programme ci dessous et expliquez la politique optimale trouvee

In [13]:
etats = ('LD_L', 'LD_M', 'LD_H', 'LC_L', 'LC_M', 'LC_H', 'CD_L','CD_M', 'CD_H', 'CC_L', 'CC_M', 'CC_H', 'O')

In [14]:
import numpy as np
#on suppose qu'aprés le nettoyage, l'état de la batterie se dégrade à chaque fois, on peut passer de batterie moyenne à faible par exemple aprés avoir effectué un nettoyage
#lorsqu'on effectue l'action charger on passe directement à un état supperieur de la batterie sauf lorsqu'on est dans un état de batterie H, on reste dans l'état de la batterie H
#lorsqu'on effectue l'action nettoyer on passe directement à un état inferrieur de la batterie par exemple un état LD_L en nettoie, on passe à l'état hors tension, de medium à low et de high à medium.
def create_transition_matrices_batterie_levels(rd, rc, costcrash,eps, e, pDust):
    P =np.zeros((2,13, 13))
    P_nettoyer = np.zeros((13, 13))
    P_charger = np.zeros((13, 13))

    # Transition probabilities for 'nettoyer'
    P_nettoyer[0] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]  # LD_L
    P_nettoyer[1] = [eps, 0, 0, 1 - eps - e, 0, 0, 0, 0, 0, 0, 0, 0, e]  # LD_M
    P_nettoyer[2] = [0, eps, 0, 0, 1 - eps - e, 0, 0, 0, 0, 0, 0, 0, e]  # LD_H

    P_nettoyer[3] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]  # LC_L
    P_nettoyer[4] = [0, 0, 0, 1 - e, 0, 0, 0, 0, 0, 0, 0, 0, e]  # LC_M
    P_nettoyer[5] = [0, 0, 0, 0, 1 - e, 0, 0, 0, 0, 0, 0, 0, e]  # LC_H

    P_nettoyer[6] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]  # CD_L
    P_nettoyer[7] = [0, eps, 0, 1 - eps, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # CD_M
    P_nettoyer[8] = [0, 1 - eps, 0, 0, eps, 0, 0, 0, 0, 0, 0, 0, 0]  # CD_H

    P_nettoyer[9] = [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # CC_L
    P_nettoyer[10] = [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]  # CC_M
    P_nettoyer[11] = [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]  # CC_H

    P_nettoyer[12] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]  # O

    # Transition probabilities for 'charger'
    P_charger[0] = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]  # LD_L
    P_charger[1] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]  # LD_M
    P_charger[2] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]  # LD_H

    P_charger[3] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]  # LC_L -> CC_M
    P_charger[4] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]  # LC_M -> CC_H
    P_charger[5] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]  # LC_H -> CC_H

    P_charger[6] = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]  # CD_L -> CD_M
    P_charger[7] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]  # CD_M -> CD_H
    P_charger[8] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]  # CD_H

    P_charger[9] = [0, 0, 0, 0, 0, 0, 0, pDust, 0, 0, 1 - pDust, 0, 0]  # CC_L -> CC_M
    P_charger[10] = [0, 0, 0, 0, 0, 0, 0, 0, pDust, 0, 0, 1 - pDust, 0]  # CC_M -> CC_H
    P_charger[11] = [0, 0, 0, 0, 0, 0, 0, 0, pDust, 0, 0, 1 - pDust, 0]  # CC_H

    P_charger[12] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]  # O

    P[0] = P_nettoyer
    P[1] = P_charger

    R = np.zeros((13, 2))

    # Assign rewards for `nettoyer`
    R[0:3, 0] = rd  # LD states
    R[3:6, 0] = rc  # LC states
    R[6:9, 0] = rd  # CD states
    R[9:12, 0] = rc  # CC states
    R[12, 0] = costcrash  # O state

    # Assign rewards for `charger`
    R[0:3, 1] = rd  # LD states
    R[3:6, 1] = rc  # LC states
    R[6:9, 1] = rd # CD states
    R[9:12, 1] = rc  # CC states
    R[12, 1] = costcrash  # O state
    return P,R

In [15]:
# adapter la fonction display pour afficher les états de la batterie
def display(policy):
    acts = [actions[i] for i in policy]

    print("Policy:")
    print("LD_L: {}".format(acts[0]))
    print("LD_M: {}".format(acts[1]))
    print("LD_H: {}".format(acts[2]))
    print("LC_L: {}".format(acts[3]))
    print("LC_M: {}".format(acts[4]))
    print("LC_H: {}".format(acts[5]))
    print("CD_L: {}".format(acts[6]))
    print("CD_M: {}".format(acts[7]))
    print("CD_H: {}".format(acts[8]))
    print("CC_L: {}".format(acts[9]))
    print("CC_M: {}".format(acts[10]))
    print("CC_H: {}".format(acts[11]))
    print("O: {}".format(acts[12]))

#adapter run pour prendre en compte les états de la batterie
def run(rd, rc, costcrash, e, eps, pDust):
    if (e+eps)>1:
      print("Mauvaises valeurs pour e: {} et eps: {} la somme doit être <= 1".format(e, eps))
    elif (pDust > 1):
      print("Mauvaises valeurs pour pDust: {} la valeur doit être <= 1".format(pDust))
    elif (pDust < 0) or (e < 0) or (eps < 0):
      print("Mauvaises valeurs pour e: {}, eps: {} et pDust: {} les probabilités sont positives. <= 1".format(e, eps, pDust))
    elif (rd >= rc) or (rd != costcrash):
      print("Mauvaises valeurs pour rd: {}, rc: {} et costcrash: {} => rc > rd et rd = costcrash.".format(rd, rc, costcrash))
    else:
      P,R = create_transition_matrices_batterie_levels(rd, rc, costcrash, e, eps, pDust)
      #pi = mdptoolbox.mdp.ValueIterationGS(P, R, 0.9, 0.01)
      pi = mdptoolbox.mdp.PolicyIteration(P, R, 0.9)

      #pi.setVerbose()
      pi.run()
      display(pi.policy)
      pi.V
      del pi

In [16]:
# appliquer le modele sur les états de la batterie
e, eps, pDust = 0.4, 0.05, 0.8
rd, rc=-10, 10
print("rd = {}, rc = {}, costcrash = {}, e = {}, eps = {}, pDust = {},".format(rd, rc, costcrash, e, eps, pDust))

run(rd, rc, costcrash, e, eps, pDust)

rd = -10, rc = 10, costcrash = -10, e = 0.4, eps = 0.05, pDust = 0.8,
Policy:
LD_L: charger
LD_M: nettoyer
LD_H: nettoyer
LC_L: charger
LC_M: charger
LC_H: charger
CD_L: charger
CD_M: nettoyer
CD_H: nettoyer
CC_L: nettoyer
CC_M: nettoyer
CC_H: nettoyer
O: nettoyer
